In [1]:
import sys
sys.path.append("/Users/leo/projects/generalization_lipschitz")

In [2]:
from robustness.split_partitions import *

In [3]:
import torch
import torchvision

In [4]:
train_dataset = torchvision.datasets.CIFAR10(root="cifar_train", train=True,
                                      transform=torchvision.transforms.Compose(
                                          [
                                              torchvision.transforms.RandomHorizontalFlip(),
                                              torchvision.transforms.ToTensor()
                                          ]
                                      ),
                                       download=True
                                      )

Files already downloaded and verified


In [5]:
val_dataset = torchvision.datasets.CIFAR10(root="cifar_val", train=False,
                                      transform=torchvision.transforms.Compose(
                                          [
                                              torchvision.transforms.RandomHorizontalFlip(),
                                              torchvision.transforms.ToTensor()
                                          ]
                                      ),
                                       download=True
                                      )

Files already downloaded and verified


In [6]:
centroids = select_partition_centroid(100, train_dataset)

In [7]:
indices = assign_partition(val_dataset, centroids)
train_indices = assign_partition(train_dataset, centroids)

In [8]:
model_checkpoint = torch.load("resnet18.pth.tar", map_location="cpu")

In [9]:
from model._resnet import _resnet, BasicBlock

In [11]:
model = _resnet(block=BasicBlock, layers=[2,2,2,2], norm_layer=None, num_classes=10)

In [12]:
model.load_state_dict(model_checkpoint['state_dict'])

<All keys matched successfully>

In [ ]:
calculate_robustness(model=model, train_dataset=train_dataset, test_dataset=val_dataset, train_indices = train_indices, test_indices=indices, loss_func=torch.nn.CrossEntropyLoss(reduction='none'))